In [1]:
from parsing import load_problem, load_solution
from graph_representation import Graph
import time

In [2]:
# weryfikuje poprawność sortowania
def verify_toposort(T, G):
    for i in xrange(len(T)):
        v = T[i]
        for s in G.graph[v].successors:
            s = (v[0] + s[0],) + s[1:]
            if s in T[:i]:
                return False
    return True

In [3]:
# m_from, m_to - skąd zabieramy, gdzie wsadzamy
# op - zabierana operacja, para (nr_maszyny, nr_operacji)
# pos - pozycja do wsadzenia op na maszynę m_to

# tutaj numery maszyn idą od 0
def lower_bound(G, m_from, op, m_to, pos):
    def alpha(k,i):
        if k != m_from:
            return G.solution[k][i]
        if i < 0:
            i += len(G.solution[k]) - 1
        if solution[k].index(op) > i:
            return G.solution[k][i]            
        return G.solution[k][i+1]
    
    # jeśli longest_path nie istnieje, to powinien zwrócić None
    # te drogi poniżej są zawarte w H1 (pierwszej składowej G), więc
    # można by je liczyć trochę szybciej (tylko w H1)
    
    def R(k,opr):
        return G.longest_path((0,) + alpha(k,1), (0,) + opr)
    
    def Q(k,opr):
        return G.longest_path((0,) + opr, (0,) + alpha(k,-1))

    def LB(l): # 0 po to, żeby mieć jakąś liczbę, gdy drogi nie istnieją
        op_old_weight = G.graph[(0,) + op].weight
        op_new_weight = G.problem[op[0]-1][op[1]-1][m_to]
        
        maxR = max(R(l, alpha(m_to, pos-1)), R(l, op) - op_old_weight, 0)
        maxQ = max(Q(l, alpha(m_to, pos)),   Q(l, op) - op_old_weight, 0)
        return maxR + maxQ + op_old_weight # a może tu ma być op_new_weight?
    
    return max([LB(l) for l in xrange(len(solution))])

# co jeśli m_from == m_to (czy w ogóle może tak być?) - alpha(m_to, pos) może nie istnieć

In [4]:
n, m, problem = load_problem("instances/Barnes_mt10c1.fjs")
solution = load_solution("solutions/Barnes_mt10c1.txt")

G = Graph(n, m, problem, solution)
G.graph[(3,1,4)].successors

[(0, 2, 5), (0, 1, 5)]

In [6]:
G.graph[(3,1,4)].weight

36

In [10]:
len([s for s in G.vertices if s[1] == 1])

120